In [63]:
import numpy
import meshio
import pathlib
from scipy.spatial.transform import Rotation
import meshplot

In [64]:
data_dir = pathlib.Path("../meshes/graspsim/")
assert(data_dir.exists())

In [65]:
with open(data_dir / "cube_3cm_config.txt") as f:
    lines = f.readlines()
print("".join(lines))

script grasp
shapes input 5
cube_3cm.msh 0.000000 1.600000 -0.000000 -0.024541 0.999699 -0.010000 0.000000 0.000000 -1.000000 -0.999699 -0.024541 -0.000000 100 100 100 material 111 1e10 0.3
round_pad.obj -1.880502 1.440331 1.621834 0.050098 -0.076310 0.995825 -0.998741 -0.001131 0.050158 -0.002702 -0.997084 -0.076270 80.000000 80.000000 80.000000 material 2300 100000000.000000 0.45 DBC -3 -3 0 3 3 0 1.244781 0.062697 -0.095338 0 0 0
cube.obj -2.496523 4.911693 2.417238 0.050098 -0.076310 0.995825 -0.998741 -0.001131 0.050158 -0.002702 -0.997084 -0.076270 3.750000 1.500000 0.200000 linearVelocity 1.244781 0.062697 -0.095338
round_pad.obj 3.098790 1.691122 1.242676 0.050098 0.076310 -0.995825 -0.998741 0.001131 -0.050158 -0.002702 0.997084 0.076270 80.000000 80.000000 80.000000 material 2300 100000000.000000 0.45 DBC -3 -3 0 3 3 0 -1.244781 -0.062697 0.095338 0 0 0
cube.obj 3.364129 5.210945 0.466184 0.050098 0.076310 -0.995825 -0.998741 0.001131 -0.050158 -0.002702 0.997084 0.076270 3.7

In [66]:
params={
    "meshes": [],
    "obstacles": []
}

def parse_shape_line(line):
    tokens = line.split()
    mesh = {}
    mesh["mesh"] = str(data_dir / tokens[0])
    mesh["position"] = [float(t) for t in tokens[1:4]]
    R = numpy.array([float(t) for t in tokens[4:13]]).reshape(3, 3)
    # mesh["rotation"] = (numpy.array(R.as_euler("xyz")) * 180/numpy.pi).tolist()
    mesh["rotation"] = R
    mesh["scale"] = [float(t) for t in tokens[13:16]]
    if pathlib.Path(tokens[0]).suffix == ".msh":
        params["meshes"].append(mesh)
    else:
        params["obstacles"].append(mesh)
    print(tokens[16:])

i = 0
while i < len(lines):
    line = lines[i].strip()
    tokens = line.split()
    i += 1
    if tokens[0] == "shapes":
        num_shapes = int(tokens[2])
        for _ in range(num_shapes):
            line = lines[i].strip()
            parse_shape_line(line)
            i += 1
    elif tokens[0] == "selfFric":
        params["mu"] = float(tokens[1])
    elif tokens[0] == "time":
        params["tend"] = float(tokens[1])
        params["dt"] = float(tokens[2])
    elif tokens[0] == "ground":
        pass # TODO: Add ground plane to obstacles
    
print(params)

['material', '111', '1e10', '0.3']
['material', '2300', '100000000.000000', '0.45', 'DBC', '-3', '-3', '0', '3', '3', '0', '1.244781', '0.062697', '-0.095338', '0', '0', '0']
['linearVelocity', '1.244781', '0.062697', '-0.095338']
['material', '2300', '100000000.000000', '0.45', 'DBC', '-3', '-3', '0', '3', '3', '0', '-1.244781', '-0.062697', '0.095338', '0', '0', '0']
['linearVelocity', '-1.244781', '-0.062697', '0.095338']
{'meshes': [{'mesh': '../meshes/graspsim/cube_3cm.msh', 'position': [0.0, 1.6, -0.0], 'rotation': array([[-0.024541,  0.999699, -0.01    ],
       [ 0.      ,  0.      , -1.      ],
       [-0.999699, -0.024541, -0.      ]]), 'scale': [100.0, 100.0, 100.0]}], 'obstacles': [{'mesh': '../meshes/graspsim/round_pad.obj', 'position': [-1.880502, 1.440331, 1.621834], 'rotation': array([[ 0.050098, -0.07631 ,  0.995825],
       [-0.998741, -0.001131,  0.050158],
       [-0.002702, -0.997084, -0.07627 ]]), 'scale': [80.0, 80.0, 80.0]}, {'mesh': '../meshes/graspsim/cube.obj

In [71]:
for i, mesh in enumerate(params["meshes"] + params["obstacles"]):
    M = meshio.read(mesh["mesh"])
    M.points = (
        M.points 
        @ numpy.diag(mesh["scale"]).T
        # @ mesh["rotation"].T
        # + numpy.array(mesh["position"])
    )
    print(mesh["rotation"])
    print(mesh["position"])
    M.point_data={}
    M.cell_data={}
    M.write(data_dir / f"{i}.ply", file_format="ply")




[0.0, 1.6, -0.0]


Warning: cell_type "tetra" is not supported by PLY format - skipping

[-1.880502, 1.440331, 1.621834]


Warning: PLY doesn't support 64-bit integers. Casting down to 32-bit.

[-2.496523, 4.911693, 2.417238]


Warning: PLY doesn't support 64-bit integers. Casting down to 32-bit.

[3.09879, 1.691122, 1.242676]


Warning: PLY doesn't support 64-bit integers. Casting down to 32-bit.

[3.364129, 5.210945, 0.466184]


Warning: PLY doesn't support 64-bit integers. Casting down to 32-bit.